In [1]:
import  pandas as pd
import numpy as np
df = pd.read_csv('merged_df_abstract_preprocess_all.csv')
df = df.dropna(subset=['processed_abstract'])

C:\Users\cshch\AppData\Local\Temp\ipykernel_20660\2488393254.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df_abstract_preprocess_all.csv')


In [2]:
print(df.shape)
print(df.columns)

(311557, 20)
Index(['Title', 'Abstract', 'Disciplines', 'Keywords', 'Source',
       'Disciplines_split', 'Astronomy', 'Biology', 'Chemistry',
       'Computer Science', 'Economics', 'Education & instruction',
       'Electrical Engineering', 'European & international law', 'History',
       'Mathematics', 'Physics', 'Social Sciences', 'Statistics',
       'processed_abstract'],
      dtype='object')


In [3]:
# Preprocessing
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Vectorize the Text Data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_abstract'])
y = df['Disciplines']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 3 RandomForest
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

model_mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
model_mlp.fit(X_train, y_train)

joblib.dump(model_mlp, 'model/4_Model_MLP.joblib')
y_pred = model_mlp.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

C:\Users\cshch\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy: 0.8580209269482604
Classification Report:
                                                      precision    recall  f1-score   support

                                          Astronomy       0.79      0.74      0.77      1887
                                  Astronomy/Physics       0.82      0.83      0.82      2640
                                            Biology       0.85      0.74      0.79       292
                                  Chemistry/Physics       0.63      0.55      0.59       291
                                   Computer Science       0.91      0.92      0.92     24139
            Computer Science/Electrical Engineering       0.57      0.54      0.55      2745
Computer Science/Electrical Engineering/Mathematics       0.44      0.36      0.40       539
                       Computer Science/Mathematics       0.44      0.39      0.41       687
                        Computer Science/Statistics       0.30      0.23      0.26       404
                

In [7]:
import numpy as np
import pandas as pd

feature_names = vectorizer.get_feature_names_out()
weights_input_hidden = model_mlp.coefs_[0]
weights_hidden_output = model_mlp.coefs_[1]

n_top_keywords = 100
keywords_dict = {}

for i, label in enumerate(model_mlp.classes_):
    class_weights = weights_hidden_output[:, i]
    top_keywords_idx = np.argsort(class_weights)[-n_top_keywords:][::-1]
    top_keywords = [(feature_names[idx], class_weights[idx]) for idx in top_keywords_idx]
    keywords_dict[label] = top_keywords

keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
keywords_df.to_csv("keywords_from_model/4_keywords_MLP.csv")
